In [ ]:
import getpass
import os
import requests
import json
import time
import faiss
import datetime

from langchain_gigachat.chat_models import GigaChat
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryMemory, VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain_community.embeddings import GigaChatEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage, SystemMessage
from langchain.tools import tool
from langchain_community.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain.tools import Tool
from langchain.agents import AgentExecutor
from langchain_gigachat.tools.giga_tool import giga_tool
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

from pydantic import BaseModel, Field
from dotenv import load_dotenv
from dateutil import parser

In [9]:
# Load environment variables
load_dotenv('.env')
giga_key = os.environ.get("SB_AUTH_DATA")
openweather_key = os.environ.get("OPENWEATHER_API_KEY")

# 1. Memory

Define your own class implementing a simple LLM-based chatbot. You need to use at least three memory types (langchain.memory), which are set as one argument in the ```init``` definition. If the memory type has any parameters, you also need to define them as arguments in the ```init``` definition. You also need to define a ```run``` method implementing the main conversation loop, and a ```print_memory``` method to print out what exactly the memory consists of.

In [6]:
class SimpleChatBot:
    def __init__(self, llm, memory_type, memory_window_size=5, max_token_limit=100):
        self.llm = llm
        self.memory = self._create_memory(memory_type, memory_window_size, max_token_limit)
        self.conversation = ConversationChain(llm=llm, memory=self.memory, verbose=False)

    def _create_memory(self, memory_type, memory_window_size, max_token_limit):
        if memory_type == "buffer":
            return ConversationBufferMemory()
        elif memory_type == "window":
            return ConversationBufferWindowMemory(k=memory_window_size)
        elif memory_type == "summary":
            return ConversationSummaryMemory(llm=self.llm, max_token_limit=max_token_limit)
        else:
            raise ValueError(f"Неизвестный тип памяти: {memory_type}")

    def _respond(self, user_input):
        response = self.conversation.predict(input=user_input)
        return response

    def print_memory(self):
        print("Содержимое памяти:")
        if isinstance(self.memory, ConversationBufferMemory) or isinstance(self.memory, ConversationBufferWindowMemory):
            print(self.memory.buffer)
        elif isinstance(self.memory, ConversationSummaryMemory):
            print(f"Последний обмен: {self.memory.buffer}")

    def run(self):
        print("This is a simple chat bot:")
        while True:
            user_input = input("Вы: ")
            if user_input.lower() == "выход":
                break
            response = self._respond(user_input)
            print(f"Бот: {response}")


Now let's check how it works with each type of memory

In [10]:
giga = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)
chat = SimpleChatBot(giga, 'summary', memory_window_size=2)
chat.run()
chat.print_memory()

/var/folders/7l/0qq42vs92fj4kw3j3dsl2ttw0000gn/T/ipykernel_6338/1087390494.py:5: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  self.conversation = ConversationChain(llm=llm, memory=self.memory, verbose=False)


This is a simple chat bot:
Бот: Привет! Как у тебя дела?
Бот: Отлично! Чем занимаешься?
Бот: AI: Какие задачи или проекты вас сейчас занимают? Может быть, есть что-то интересное или сложное, с чем вы столкнулись?
Бот: Кажется, ты шутишь, говоря, что всё легко, но я рад, что у тебя такие интересные задачи! Расскажи, над чем именно ты сейчас работаешь? Какие технологии или инструменты используешь?
Содержимое памяти:
Последний обмен: Привет! AI ответил на приветствие и поинтересовался, как у меня дела. Я ответил, что все хорошо. AI спросил, чем я занимаюсь. Я сказал, что работаю. AI спросил о текущих задачах или проектах. Я ответил, что передо мной стоят интересные или сложные задачи. Человек ответил, что все легко, несмотря на сложность задач, и заинтересовался, какие технологии я использую.


In [12]:
giga = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)
chat = SimpleChatBot(giga, 'window', memory_window_size=2)
chat.run()
chat.print_memory()

This is a simple chat bot:
Бот: Привет! Как у тебя дела?
Бот: Похоже, что вводная часть разговора оборвалась. Если хочешь, можем продолжить с того места, где остановились, или начать новый диалог.
Бот: Хорошо, начнем сначала! 

Привет! Как у тебя дела?
Бот: Отлично! Рад слышать, что у тебя все хорошо. Чем занимаешься?
Бот: Кажется, в нашем разговоре возникла пауза. Если у тебя есть какие-то конкретные вопросы или темы для обсуждения, я с радостью помогу!
Бот: Похоже, разговор подошел к концу. Если захочешь снова пообщаться или обсудить какую-нибудь тему, я всегда здесь! До скорых встреч! 😊
Содержимое памяти:
Human: да опять работаю
AI: Кажется, в нашем разговоре возникла пауза. Если у тебя есть какие-то конкретные вопросы или темы для обсуждения, я с радостью помогу!
Human: пока!
AI: Похоже, разговор подошел к концу. Если захочешь снова пообщаться или обсудить какую-нибудь тему, я всегда здесь! До скорых встреч! 😊


In [14]:
giga = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)
chat = SimpleChatBot(giga, 'buffer', memory_window_size=2)
chat.run()
chat.print_memory()

This is a simple chat bot:
Бот: Привет! Как у тебя дела?
Бот: Похоже, что разговор прервался. Если у тебя есть еще вопросы или хочешь продолжить беседу, я готов помочь!
Бот: Кажется, что в этом отрывке разговора чего-то не хватает. Возможно, стоит добавить немного контекста или уточнить, о чем идет речь. Если нужно, могу помочь развить тему или ответить на новый вопрос.
Бот: Конечно, можем общаться столько, сколько тебе хочется! Я здесь, чтобы поддерживать интересный и полезный диалог. Какой у тебя следующий вопрос или тема для обсуждения?
Содержимое памяти:
Human: привет
AI: Привет! Как у тебя дела?
Human: хорошо
AI: Похоже, что разговор прервался. Если у тебя есть еще вопросы или хочешь продолжить беседу, я готов помочь!
Human: еще раз привет
AI: Кажется, что в этом отрывке разговора чего-то не хватает. Возможно, стоит добавить немного контекста или уточнить, о чем идет речь. Если нужно, могу помочь развить тему или ответить на новый вопрос.
Human: будем долго разговаривать?
AI: Коне

Please make a short report abount differences between used memory types

Report:

- **ConversationBufferMemory**: Сохраняет всю историю диалога. Полезно для полного контекста.
- **ConversationBufferWindowMemory**: Хранит только последние k сообщений. Подходит для учета недавних взаимодействий.
- **ConversationSummaryMemory**: Создает резюме диалога. Идеально для сохранения общего контекста без деталей.


# 2. Using tools and agents

## 2.1 Using tools and API

Create your own tool based on the langchain.tools library to interact with a public OpenWeather API. This tool will receive data from the API and return it as a readable result for the user.


OpenWeather API URL: https://api.openweathermap.org/data/2.5/weather?q={city}&appid={openweather_key}&units=metric 

[How to get OpenWeather API key](https://docs.google.com/document/d/1vbi8QKqMZqZoCReIzpmEB_2mHsrbmXPlyGngE3jeDDw/edit)


In [15]:
few_shot_examples_weather = [
    {
        "request": "Какая погода сейчас в Ставрополе?",
        "params": {"city": "Ставрополь"},
    }
]

class WeatherResult(BaseModel):
    status: str = Field(description="Статус получения погоды")
    message: str = Field(description="Сообщение о погоде")

@giga_tool(few_shot_examples = few_shot_examples_weather)
def get_weather(city: str) -> WeatherResult:
    """Получает текущую погоду для указанного города."""
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={openweather_key}&units=metric"
    response = requests.get(url)
    
    if response.status_code != 200:
        return WeatherResult(status="error", message=f"Ошибка при получении данных о погоде: {response.status_code}")
    
    data = response.json()
    timestamp = data["dt"]
    current_date = str(datetime.datetime.fromtimestamp(timestamp).strftime('%d-%m-%Y'))
    weather_description = data["weather"][0]["description"]
    temperature = data["main"]["temp"]
    feels_like = data["main"]["feels_like"]
    humidity = data["main"]["humidity"]
    wind_speed = data["wind"]["speed"]
    
    result = f"""Погода в городе {city} на {current_date}:
                 - Описание: {weather_description}
                 - Температура: {temperature}°C
                 - Ощущается как: {feels_like}°C
                 - Влажность: {humidity}%
                 - Скорость ветра: {wind_speed} м/с"""
    
    return WeatherResult(status="success", message=result)

class OpenWeatherAPITool:
    def __init__(self, llm, agent_function):
        functions = [agent_function]
        self.llm = llm.bind_functions(functions)
        
        self.agent = create_react_agent(llm, 
                                        functions,
                                        state_modifier="Ты помощник, который предоставляет информацию о погоде.")
        
    def run(self, user_input):
        resp = self.agent.invoke({"messages": [HumanMessage(content=user_input)]})
        bot_answer = resp['messages'][-1].content
        print("\033[93m" + f"Bot: {bot_answer}" + "\033[0m")

Let's check it

In [16]:

giga_pro = GigaChat(credentials=giga_key, model="GigaChat-Pro", timeout=30, verify_ssl_certs=False)

openwheatertool = OpenWeatherAPITool(giga_pro, get_weather)
user_input = "Какая погода сейчас в Ставрополе?"
openwheatertool.run(user_input)

Bot: Погода в городе Ставрополь на 24-03-2025:
                 - Описание: ясно
                 - Температура: 2.16°C
                 - Ощущается как: -2.22°C
                 - Влажность: 93%
                 - Скорость ветра: 5 м/с


## 2.2. Multi agents

Create a multi-agent system where each agent is responsible for a specific task in the travel planning process. For example, one agent is responsible for searching for flights, another for booking hotels, and a third for finding the weather at the destination.

Requirements:

- Use three or more GigaChat-based agents to interact with each other.
- The first agent is responsible for searching for flights (using ```get_url_booking_tickets``` function).
- The second agent is responsible for booking hotels (using ```get_url_booking_hotels``` function).
- The third agent collects weather information for the destination (using a real API, such as OpenWeather). You can use the function from the previous task (for simplify, here you can give a current weather, not a forecast for the specific date)

In [17]:
def get_geoid(city: str) -> str:
    url_base = 'https://suggest-maps.yandex.ru/suggest-geo'
    params = {'search_type': 'tune', 'v': '9', 'results': 1, 'lang': 'ry_RU', 'callback': 'json'}
    params['part'] = city
    r = requests.get(url_base, params=params)
    if r.ok:
        r_text = r.text
        r_json = r_text[5: len(r_text)-1]
        res_json = json.loads(r_json)
        res = res_json['results'][0]['geoid']
    else:
        res = ''
    return str(res)


class HotelBookingResult(BaseModel):
    status: str = Field(description="Статус бронирования отеля")
    message: str = Field(description="Информация о бронировании")

@giga_tool
def get_url_booking_hotels(date_in_str: str, date_out_str: str, city: str) -> HotelBookingResult:
    """Получает ссылку для бронирования отеля в указанном городе на указанные даты."""
    date_in = parser.parse(date_in_str)
    date_out = parser.parse(date_out_str)
    if date_in is None:
        date_in = datetime.datetime.now()
    if date_out is None:
        date_out = datetime.datetime.now() + datetime.timedelta(days=1)
    geoid = get_geoid(city)
    url = 'https://travel.yandex.ru/hotels/search/?' 
    params = {'adults': '2', 'checkinDate': date_in.strftime('%Y-%m-%d'), 'checkoutDate': date_out.strftime('%Y-%m-%d'), 'childrenAges': '0', 'geoId': geoid}
    for item in params:
        url += '&' + item + '=' + params[item]
    result = f'Ссылка для бронирования отеля: {url} в городе {city} на даты {date_in_str} / {date_out_str}'
    return HotelBookingResult(status="success", message=result)


class TicketBookingResult(BaseModel):
    status: str = Field(description="Статус бронирования билетов")
    message: str = Field(description="Информация о билетах")

@giga_tool
def get_url_booking_tickets(city_from: str, city_to: str, date_in_str: str, date_out_str: str) -> TicketBookingResult:
    """Получает ссылку для бронирования авиабилетов между указанными городами на указанные даты."""
    date_in = parser.parse(date_in_str)
    date_out = parser.parse(date_out_str)
    if date_in is None:
        date_in = datetime.datetime.now()
    if date_out is None:
        date_out = datetime.datetime.now() + datetime.timedelta(days=1)
    fromid = get_geoid(city_from)
    toid = get_geoid(city_to)
    url = 'https://travel.yandex.ru/avia/search/result/?' 
    params = {'adults_seats': '2', 'fromId': 'c' + fromid, 'klass': 'economy', 'oneway': '2', 'return_date': date_out.strftime('%Y-%m-%d'), 'toId': 'c' + toid, 'when': date_in.strftime('%Y-%m-%d')}
    for item in params:
        url += '&' + item + '=' + params[item]
    result = f'Ссылка для заказа билетов: {url} из {city_from} в {city_to} на даты {date_in_str} / {date_out_str}'
    return TicketBookingResult(status="success", message=result)


class MultiAgent:
    def __init__(self, llm, agent_function_weather, agent_function_hotels, agent_function_tickets):
        self.llm = llm
        
        weather_functions = [agent_function_weather]
        self.weather_llm = llm.bind_functions(weather_functions)
        self.weather_agent = create_react_agent(
            self.weather_llm,
            weather_functions,
            state_modifier="Ты помощник, который предоставляет информацию о погоде в городе назначения."
        )
        
        hotel_functions = [agent_function_hotels]
        self.hotel_llm = llm.bind_functions(hotel_functions)
        self.hotel_agent = create_react_agent(
            self.hotel_llm,
            hotel_functions,
            state_modifier="Ты помощник, который помогает с бронированием отелей."
        )
        
        ticket_functions = [agent_function_tickets]
        self.ticket_llm = llm.bind_functions(ticket_functions)
        self.ticket_agent = create_react_agent(
            self.ticket_llm,
            ticket_functions,
            state_modifier="Ты помощник, который помогает с поиском и бронированием авиабилетов."
        )
        
        # Создаем координатора
        self.coordinator_llm = llm
        
    def _create_agent(self, tools, system_prompt):
        functions = tools
        llm_with_functions = self.llm.bind_functions(functions)
        agent = create_react_agent(
            llm_with_functions,
            functions,
            state_modifier=system_prompt
        )
        return agent
    
        
    def run(self, user_input: str):
        coordinator_prompt = f"""
        Ты помощник по организации путешествий. Проанализируй запрос пользователя и выдели следующую информацию:
        - Город отправления
        - Город назначения
        - Дата прибытия
        - Дата отъезда

        Если какие-то данные отсутствуют, примени разумные предположения на основе контекста.

        ВАЖНО: Все даты должны быть представлены в формате дд-мм-гггг (например, 01-06-2023).
        Если дата в запросе представлена в другом формате, преобразуй её в формат дд-мм-гггг.

        Верни ответ строго в формате JSON:
        {{
            "city_from": "название города отправления",
            "city_to": "название города назначения",
            "date_in": "дата прибытия в формате дд-мм-гггг",
            "date_out": "дата отъезда в формате дд-мм-гггг"
        }}

        Запрос пользователя: {user_input}
        """
        
        coordinator_response = self.coordinator_llm.invoke(coordinator_prompt)
        
        try:
            # Извлекаем JSON из ответа координатора
            import re
            import json
            
            # Найдем JSON-структуру в ответе
            json_match = re.search(r'({[\s\S]*})', coordinator_response.content)
            if json_match:
                json_str = json_match.group(1)
                travel_data = json.loads(json_str)
                
                city_from = travel_data.get("city_from", "Москва")
                city_to = travel_data.get("city_to", "Санкт-Петербург")
                date_in = travel_data.get("date_in", "01-06-2023")
                date_out = travel_data.get("date_out", "10-06-2023")
            else:
                raise ValueError("JSON не найден в ответе")
        except Exception as e:
            print(f"Ошибка при обработке ответа: {e}")
            # Если не удалось извлечь данные, используем значения по умолчанию
            city_from = "Москва"
            city_to = "Санкт-Петербург"
            date_in = "01-06-2023"
            date_out = "10-06-2023"
        
        # Создаем подробное резюме о полученных данных
        print(f"Планируем путешествие: из {city_from} в {city_to} с {date_in} по {date_out}")
        
        weather_query = f"Какая погода в городе {city_to}?"
        weather_response = self.weather_agent.invoke({"messages": [HumanMessage(content=weather_query)]})
        weather_info = weather_response['messages'][-1].content
        
        ticket_query = f"Найди билеты из {city_from} в {city_to} на даты {date_in} и {date_out}"
        ticket_response = self.ticket_agent.invoke({"messages": [HumanMessage(content=ticket_query)]})
        ticket_info = ticket_response['messages'][-1].content
        
        hotel_query = f"Найди отели в городе {city_to} на даты {date_in} и {date_out}"
        hotel_response = self.hotel_agent.invoke({"messages": [HumanMessage(content=hotel_query)]})
        hotel_info = hotel_response['messages'][-1].content
        
        answer = f"""
        Информация о вашем путешествии из {city_from} в {city_to}:
        
        ПОГОДА:
        {weather_info}
        
        БИЛЕТЫ:
        {ticket_info}
        
        ОТЕЛИ:
        {hotel_info}
        """
        
        return answer


In [18]:
giga_pro = GigaChat(credentials=giga_key, model="GigaChat-Pro", timeout=30, verify_ssl_certs=False)

traveler = MultiAgent(giga_pro, get_weather, get_url_booking_hotels, get_url_booking_tickets)
user_input = "Организуй поездку из Пензы в Санкт-Петербург с 25.03.2025 по 30.03.2025 - отель, самолет, погода"
answer = traveler.run(user_input)
print(answer)

Планируем путешествие: из Пенза в Санкт-Петербург с 25-03-2025 по 30-03-2025

        Информация о вашем путешествии из Пенза в Санкт-Петербург:

        ПОГОДА:
        Погода в городе Санкт-Петербург на 24-03-2025:
                 - Описание: малооблачно
                 - Температура: 0.6°C
                 - Ощущается как: -3.55°C
                 - Влажность: 55%
                 - Скорость ветра: 4 м/с

        БИЛЕТЫ:
        Ссылка для заказа билетов: https://travel.yandex.ru/avia/search/result/?&adults_seats=2&fromId=c49&klass=economy&oneway=2&return_date=2025-03-30&toId=c2&when=2025-03-25 из Пенза в Санкт-Петербург на даты 25-03-2025 / 30-03-2025

        ОТЕЛИ:
        Ссылка для бронирования отеля в городе Санкт-Петербург на указанные Вами даты (25-03-2025 - 30-03-2025) следующая: https://travel.yandex.ru/hotels/search/?&adults=2&checkinDate=2025-03-25&checkoutDate=2025-03-30&childrenAges=0&geoId=2.
        
